# Objectifs

* ~~capture multiples photos sur une durée précise~~ capture video ok -> envoi numpy array au cnn
* ~~se séparer de l'écriture (=enregistrement) en local de ces photos (pas de photo.jpeg en local)~~ os.remove fonctionnel
* ~~parvenir à écrire de multiples frames ou les regrouper dans un batch à passer au cnn (sans écrire le batch en local)~~ inclus en vidéo
* ~~avoir un simple script module_1 qui prenne en input la photo, et la restitue en output~~ ok dummycnn.py
* **ignorer deepface: fouiller les outils openCV pour détecter la présence d'un visage** -> nice-to-have
* ~~supprimer l'enregistrement vidéo et simplifier la cellule openCV~~ ok: déclenchement/interruption manuel (lien avec case app début/fin exercice)
* collectiviser les réponses CNN, extraire la plus fréquente, envoyer au LLM cette réponse AVEC une limite dans le temps de nb de requêtes

In [1]:
import numpy
import cv2
from PIL import Image
import time
from collections import deque

In [ ]:
import cnn
cap = cv2.VideoCapture(0)
prev_frame_time = 0
new_frame_time = 0
llminput = []

while( cap.isOpened() ):
    ret, frame = cap.read()
    if ret == True:
        frame = cv2.flip(frame,1)

        #display fps
        gray = frame
        font = cv2.FONT_HERSHEY_SIMPLEX
        new_frame_time = time.time()
        fps = 1/(new_frame_time - prev_frame_time)
        prev_frame_time = new_frame_time
        fps = int(fps)
        fps = str(fps)
        cv2.putText(gray, fps, (7, 70), font, 3, (100, 255, 0), 3, cv2.LINE_AA)

        #transfo image pour envoi au CNN
        pilimage = Image.fromarray(frame).convert("RGB")
        cnnresponse = cnn.getemotion(pilimage)
        print(cnnresponse)
        llminput.append(cnnresponse)
        cv2.imshow('frame' , frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break

cap.release()
cv2.destroyAllWindows()

c:\Users\maria\anaconda3\envs\deep_learning\Lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] La procédure spécifiée est introuvable'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
c:\Users\maria\anaconda3\envs\deep_learning\Lib\site-packages\torchvision\datapoints\__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future ch

others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others
others

In [4]:
testdeque = deque(llminput, maxlen=100)
max(testdeque)

'others'

In [5]:
testdeque

deque(['others',
       'others',
       'others',
       'others',
       'others',
       'others',
       'others',
       'others',
       'others',
       'others',
       'others',
       'others',
       'others',
       'others',
       'others',
       'others',
       'others',
       'others',
       'others',
       'others',
       'others',
       'others',
       'others',
       'others',
       'others',
       'others',
       'others',
       'others',
       'others',
       'others',
       'others',
       'others',
       'others',
       'others',
       'others',
       'others',
       'others',
       'others',
       'others',
       'others',
       'others',
       'others',
       'others',
       'others',
       'others',
       'others',
       'others',
       'others',
       'others',
       'others',
       'others',
       'others',
       'others',
       'others',
       'others',
       'others',
       'others',
       'others',
       'others

In [24]:
# Initialize webcam (0 = default camera)
cam = cv2.VideoCapture(0)

# Capture one frame
ret, frame = cam.read()

if ret:
    cv2.imwrite("captured_image.jpeg", frame)  
else:
    print("Failed to capture image.")

cam.release()

In [2]:
%load_ext autoreload
%autoreload 2
import module_1
module_1.random_nb_call()

2